In [6]:
from ee_ipl_uv import normalization
from ee_ipl_uv import converters

import numpy as np
import ee


ee.Initialize()
im = ee.Image('LANDSAT/LC8_L1T_TOA/LC81980332015119LGN00').select(["B2","B3","B4"])
ftcol = im.sample(numPixels=10,seed=23)
ftcol_pd = converters.eeFeatureCollectionToPandas(ftcol,["B2","B3","B4"])

weights = ftcol_pd.B4
ftcol_pd_1 = ftcol_pd[["B2","B3"]]
    
ftcol_pd_2 = ftcol_pd_1.apply(lambda x: x -np.average(x,weights=weights))

ftcol_pd_3 = ftcol_pd_1.apply(lambda x: x -np.average(x,weights=100*weights))

np.testing.assert_allclose(ftcol_pd_2, ftcol_pd_3)

In [5]:
ftcol_norm, mean, std = normalization.ComputeNormalizationFeatureCollection(ftcol,["B2","B3"],weight="B4",
                                                                           only_center_data=True)

ftcol_norm_pd = converters.eeFeatureCollectionToPandas(ftcol_norm,["B2","B3"])

np.testing.assert_allclose(ftcol_norm_pd, ftcol_pd_2)
# weights = [weight for b in bands_to_normalize]